In [38]:
import openai
import pinecone
import langchain
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain_google_genai import GoogleGenerativeAI

In [6]:
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
import os

In [8]:
def read_doc(directory):
    file_load = PyPDFDirectoryLoader(directory)
    documents = file_load.load()
    return documents

In [10]:
doc = read_doc('documents/')
len(doc)

1

In [11]:
def chunk_data(docs, chunk_size=100,chunk_overlap=20):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    docs = text_splitter.split_documents(docs)
    return docs

In [18]:
documents = chunk_data(docs=doc)
documents

[Document(metadata={'producer': 'Canva', 'creator': 'Canva', 'creationdate': '2025-08-18T06:13:31+00:00', 'title': 'RESUME IN PROCESS', 'moddate': '2025-08-18T06:13:30+00:00', 'keywords': 'DAGtNw66A_M,BAEiyhLqebQ,0', 'author': 'Hitesh', 'source': 'documents\\Hitesh_resume_fin.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='H I T E S H  K U M A R'),
 Document(metadata={'producer': 'Canva', 'creator': 'Canva', 'creationdate': '2025-08-18T06:13:31+00:00', 'title': 'RESUME IN PROCESS', 'moddate': '2025-08-18T06:13:30+00:00', 'keywords': 'DAGtNw66A_M,BAEiyhLqebQ,0', 'author': 'Hitesh', 'source': 'documents\\Hitesh_resume_fin.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='A n  A I  e n t h u s i a s t  w i t h  p r a c t i c a l  e x p e r t i s e  i n  m a c h i n e  l'),
 Document(metadata={'producer': 'Canva', 'creator': 'Canva', 'creationdate': '2025-08-18T06:13:31+00:00', 'title': 'RESUME IN PROCESS', 'moddate': '2025-08-18T06:13:30+00:00', 'ke

In [24]:
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",   # Google Gemini embedding model
    google_api_key=os.environ["GOOGLE_API_KEY"]  # use Google key
)


In [26]:
vectors = embeddings.embed_query("How are you?")
len(vectors)

768

In [ ]:
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

# Set environment variable
os.environ['PINECONE_API_KEY'] = """your pinecone api key(can't show mine)"""

# Use LangChain's PineconeVectorStore for from_documents
index_name = 'langchainvector'
vectorstore = PineconeVectorStore.from_documents(
    documents=doc,
    embedding=embeddings,
    index_name=index_name
)

In [49]:
#Cosine similarity retrieve results from the VectorDB
def retrieve_query(query, k=2):
    matching_results = vectorstore.similarity_search(query,k=k)
    return matching_results



In [51]:
from langchain.chains import RetrievalQA
from langchain_google_genai import GoogleGenerativeAI

In [ ]:
llm = GoogleGenerativeAI(
    model="gemini-2.5-flash",  # Google Gemini Pro model
    temperature=0.5,
    google_api_key=os.environ["GOOGLE_API_KEY"]  # use Google key
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(),  # you must provide a retriever now
    chain_type="stuff"
)


In [63]:
def retrieve_answers(query: str):
    response = qa.invoke({"query": query})   # <- correct call
    return response["result"]

my_query = "what is the document about?"
answer = retrieve_answers(my_query)
print(answer)


The document is a resume or curriculum vitae for Hitesh Kumar, highlighting his skills and experience in artificial intelligence, machine learning, and related fields.
